In [9]:
!pip install tensorflow

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [10]:
import tensorflow as tf
from keras.layers import Input, Lambda, Dense, Flatten,Dropout
from keras.models import Model
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [11]:
IMAGE_SIZE = [224, 224]

In [12]:
train_path='/kaggle/input/car-images2/kaggle_upload/train_resized_224'
val_path='/kaggle/input/car-images2/kaggle_upload/val_resized_224'

In [13]:
folders = glob('/kaggle/input/car-images2/kaggle_upload/train_resized_224/*')
print(len(folders))

196


In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
num_classes = 196

# Prepare the data using ImageDataGenerator
train_datagen_upsampled1= ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=[0.8, 1.2]  # Example of additional augmentation
)

# Prepare training and validation generators
train_generator_upsampled1 = train_datagen_upsampled1.flow_from_directory(
    directory=train_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

test_datagen_upsampled1= ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=[0.8, 1.2]  # Example of additional augmentation
)


test_generator_upsampled1 = test_datagen_upsampled1.flow_from_directory(
    val_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


Found 13326 images belonging to 196 classes.
Found 8041 images belonging to 196 classes.


In [17]:
import tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications import InceptionV3  
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

input_shape = (224, 224, 3)
num_classes = 196

# Loading the Pre-Trained GoogLeNet Model (InceptionV3)
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)

# Customizing the Model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)

model_googlenetUpsampled = Model(inputs=base_model.input, outputs=predictions)

# Setting the parameters to train the last 30 layers 
for layer in base_model.layers[-30:]:
    layer.trainable = True

# Compiling the Model
model_googlenetUpsampled.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
history_googlenetUpsampled = model_googlenetUpsampled.fit(
    train_generator_upsampled1,
    validation_data=test_generator_upsampled1,
    epochs=30,
    steps_per_epoch=train_generator_upsampled1.samples // train_generator_upsampled1.batch_size,
    validation_steps=test_generator_upsampled1.samples // test_generator_upsampled1.batch_size
)

Epoch 1/30


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


416/416 ━━━━━━━━━━━━━━━━━━━━ 395s 771ms/step - accuracy: 0.0073 - loss: 5.3233 - val_accuracy: 0.0388 - val_loss: 4.7780
Epoch 2/30
  1/416 ━━━━━━━━━━━━━━━━━━━━ 1:05 159ms/step - accuracy: 0.0312 - loss: 4.6364

/opt/conda/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


416/416 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.0312 - loss: 4.6364 - val_accuracy: 0.0000e+00 - val_loss: 4.8089
Epoch 3/30
416/416 ━━━━━━━━━━━━━━━━━━━━ 280s 667ms/step - accuracy: 0.0790 - loss: 4.4504 - val_accuracy: 0.1950 - val_loss: 3.4597
Epoch 4/30
416/416 ━━━━━━━━━━━━━━━━━━━━ 0s 111us/step - accuracy: 0.0938 - loss: 3.5940 - val_accuracy: 0.4444 - val_loss: 3.0735
Epoch 5/30
416/416 ━━━━━━━━━━━━━━━━━━━━ 274s 651ms/step - accuracy: 0.2561 - loss: 3.1210 - val_accuracy: 0.4076 - val_loss: 2.3084
Epoch 6/30
416/416 ━━━━━━━━━━━━━━━━━━━━ 0s 110us/step - accuracy: 0.1562 - loss: 3.0695 - val_accuracy: 0.3333 - val_loss: 3.2070
Epoch 7/30
416/416 ━━━━━━━━━━━━━━━━━━━━ 272s 648ms/step - accuracy: 0.4381 - loss: 2.1962 - val_accuracy: 0.5039 - val_loss: 1.8645
Epoch 8/30
416/416 ━━━━━━━━━━━━━━━━━━━━ 0s 119us/step - accuracy: 0.4688 - loss: 1.9126 - val_accuracy: 0.5556 - val_loss: 1.6005
Epoch 9/30
416/416 ━━━━━━━━━━━━━━━━━━━━ 272s 646ms/step - accuracy: 0.5583 - loss: 1.6466 - 

In [21]:
test_loss, test_acc = model_googlenetUpsampled.evaluate(test_generator_upsampled1)
print(f"Test accuracy: {test_acc * 100:.2f}%")

train_loss, train_acc = model_googlenetUpsampled.evaluate(train_generator_upsampled1)
print(f"Train accuracy: {train_acc * 100:.2f}%")

252/252 ━━━━━━━━━━━━━━━━━━━━ 102s 405ms/step - accuracy: 0.6761 - loss: 1.3033
Test accuracy: 68.24%
417/417 ━━━━━━━━━━━━━━━━━━━━ 180s 432ms/step - accuracy: 0.9268 - loss: 0.2491
Train accuracy: 92.34%


In [23]:
import tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications import EfficientNetB0  # InceptionV3 is a successor to GoogLeNet
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


input_shape = (224, 224, 3)
num_classes = 196


base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)

# Customizing the Model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)

model_efficientnetUpsampled = Model(inputs=base_model.input, outputs=predictions)


for layer in base_model.layers[-30:]:
    layer.trainable = True

# Compiling the Model
model_efficientnetUpsampled.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [24]:
history_efficientnetUpsampled = model_efficientnetUpsampled.fit(
    train_generator_upsampled1,
    validation_data=test_generator_upsampled1,
    epochs=30,
    steps_per_epoch=train_generator_upsampled1.samples // train_generator_upsampled1.batch_size,
    validation_steps=test_generator_upsampled1.samples // test_generator_upsampled1.batch_size
)

Epoch 1/30
416/416 ━━━━━━━━━━━━━━━━━━━━ 403s 772ms/step - accuracy: 0.0106 - loss: 5.2700 - val_accuracy: 0.0092 - val_loss: 5.2672
Epoch 2/30
416/416 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.0938 - loss: 4.5979 - val_accuracy: 0.0000e+00 - val_loss: 5.3510
Epoch 3/30
416/416 ━━━━━━━━━━━━━━━━━━━━ 278s 660ms/step - accuracy: 0.0868 - loss: 4.4747 - val_accuracy: 0.2668 - val_loss: 3.2981
Epoch 4/30
416/416 ━━━━━━━━━━━━━━━━━━━━ 0s 101us/step - accuracy: 0.2812 - loss: 3.4236 - val_accuracy: 0.5556 - val_loss: 2.8103
Epoch 5/30
416/416 ━━━━━━━━━━━━━━━━━━━━ 281s 668ms/step - accuracy: 0.2482 - loss: 3.2183 - val_accuracy: 0.4675 - val_loss: 2.1793
Epoch 6/30
416/416 ━━━━━━━━━━━━━━━━━━━━ 0s 102us/step - accuracy: 0.3438 - loss: 2.8336 - val_accuracy: 0.5556 - val_loss: 2.1761
Epoch 7/30
416/416 ━━━━━━━━━━━━━━━━━━━━ 278s 661ms/step - accuracy: 0.3997 - loss: 2.3848 - val_accuracy: 0.5476 - val_loss: 1.7588
Epoch 8/30
416/416 ━━━━━━━━━━━━━━━━━━━━ 0s 92us/step - accuracy: 0.4062 - loss: 

In [26]:
test_loss, test_acc = model_efficientnetUpsampled.evaluate(test_generator_upsampled1)
print(f"Test accuracy: {test_acc * 100:.2f}%")

train_loss, train_acc = model_efficientnetUpsampled.evaluate(train_generator_upsampled1)
print(f"Train accuracy: {train_acc * 100:.2f}%")

252/252 ━━━━━━━━━━━━━━━━━━━━ 98s 391ms/step - accuracy: 0.7603 - loss: 0.9093
Test accuracy: 77.02%
417/417 ━━━━━━━━━━━━━━━━━━━━ 175s 419ms/step - accuracy: 0.9755 - loss: 0.0913
Train accuracy: 97.52%


In [28]:
import tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# Resizing the input shape to (224,224)
input_shape = (224, 224, 3)
num_classes = 196

# Loading the Pre-Trained EfficientNetB0 Model
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)

# Customizing the Model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)

model_efficientnetUpsampled = Model(inputs=base_model.input, outputs=predictions)

# Freezing Base Model Layers
for layer in base_model.layers[-30:]:
    layer.trainable = True

# Compiling the Model
model_efficientnetUpsampled.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
checkpoint = ModelCheckpoint('model_weights_efficient_best.keras', save_best_only=True, monitor='val_loss', mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

history_efficientnetUpsampled = model_efficientnetUpsampled.fit(
    train_generator_upsampled1,
    validation_data=test_generator_upsampled1,
    epochs=30,
    steps_per_epoch=train_generator_upsampled1.samples // train_generator_upsampled1.batch_size,
    validation_steps=test_generator_upsampled1.samples // test_generator_upsampled1.batch_size,
    callbacks=[checkpoint, early_stopping, reduce_lr]
)


Epoch 1/30
416/416 ━━━━━━━━━━━━━━━━━━━━ 655s 1s/step - accuracy: 0.0095 - loss: 5.2843 - val_accuracy: 0.0075 - val_loss: 5.2734 - learning_rate: 1.0000e-04
Epoch 2/30
416/416 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.0312 - loss: 4.9706 - val_accuracy: 0.0000e+00 - val_loss: 5.3042 - learning_rate: 1.0000e-04
Epoch 3/30
416/416 ━━━━━━━━━━━━━━━━━━━━ 276s 656ms/step - accuracy: 0.2528 - loss: 3.1996 - val_accuracy: 0.4438 - val_loss: 2.3064 - learning_rate: 1.0000e-04
Epoch 6/30
416/416 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3438 - loss: 2.5821 - val_accuracy: 0.4444 - val_loss: 1.8221 - learning_rate: 1.0000e-04
Epoch 7/30
416/416 ━━━━━━━━━━━━━━━━━━━━ 272s 648ms/step - accuracy: 0.4107 - loss: 2.3764 - val_accuracy: 0.5762 - val_loss: 1.6821 - learning_rate: 1.0000e-04
Epoch 8/30
416/416 ━━━━━━━━━━━━━━━━━━━━ 0s 96us/step - accuracy: 0.4688 - loss: 1.6695 - val_accuracy: 0.5556 - val_loss: 1.8580 - learning_rate: 1.0000e-04
Epoch 9/30
416/416 ━━━━━━━━━━━━━━━━━━━━ 272s 647ms

In [ ]:
test_loss, test_acc = model_efficientnetUpsampled.evaluate(test_generator_upsampled1)
print(f"Test accuracy: {test_acc * 100:.2f}%")

train_loss, train_acc = model_efficientnetUpsampled.evaluate(train_generator_upsampled1)
print(f"Train accuracy: {train_acc * 100:.2f}%")

 54/252 ━━━━━━━━━━━━━━━━━━━━ 1:18 397ms/step - accuracy: 0.7489 - loss: 0.9295

## Final Conclusions

Tried with efficientnet and inceptionv3 model, tried several different methods , first allowing the last 10 layers to train , that didnt give the desired results, then tried with unfreezing the last 30 layers , which gave most optimum results

The highest accuracy both in terms of train and test was given by EfficientNetB0 giving 

Test accuracy: 77.02%
Train accuracy: 97.52%